In [ ]:
dir_root = '/user/centos/'
import findspark
findspark.init()

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql import functions as sf

In [ ]:
ddf = spark.createDataFrame([[np.nan, 'John'],
                             [None, 'Michael'],
                             [30., 'Andy'],
                             [19., 'Justin'],
                             [30., 'James Dr No From Russia with Love Bond']], 
                             schema = ['age', 'name'])

### *Exercise*

1. Explore the `ddf_air` DF
    - count how many NaN's you have in each column;
2. Fill the NaN with something that makes sense for each column.
3. With a UDF, capture the state in the `airport_name` column (e.g. 'NY' in 'New York, NY: John F. Kennedy International') and
4. make a new dataframe `ddf_states` with columns `airport, state`
3. Remove duplicates from ddf_states (hint: lookup `drop_duplicates()` in the docs)
3. Join `ddf_states` onto the original `ddf_air` 
7. add a column weather_condition that is 
```
'rainy' if the `weather_delay` is greather than 1200
'stormy' if in addition to this the arrival is diverted by more than 15 minutes
'bright' otherwise
```
6. Split the DF into a train and test set sorted by time cols (hint: lookup `limit()` or `randomSplit()` in the docs )

In [ ]:
# 1.
ddf_air = spark.read.load(dir_root + 'data/airlines.parquet')
ddf_air.show(5)

Columns mean:

* `arr_flights`: flights arrived
* `arr_del15`: flights delayed more than 15';
* `carrier_ct`: delayed by carrier;
* `weather_ct`: by weather;
* `nas_ct`: by national aviation system;
* `security_ct`: by security;
* `late_aircraft_ct`: by late aircraft arrival;
* `arr_cancelled`: cancelled;
* `arr_diverted`: deverted;
* `arr_delay`: total delay and then breakdown below;
* `carrier_delay`;
* `weather_delay`;
* `nas_delay`;
* `security_delay`;
* `late_aircraft_delay`.

In [ ]:
# 1a

ddf_air_length = ddf_air.count()
for col in ddf_air.columns:
    print col, ddf_air_length - ddf_air.dropna(subset=col).count()

In [ ]:
# 2
ddf_air = ddf_air.fillna(0)

In [ ]:
# 3 & 4 & 5
from pyspark.sql.types import StringType
state = sf.udf(lambda s: s.split(':')[0][-2:], StringType())
ddf_states = (ddf_air
              .withColumn('state', state(sf.col('airport_name')))
              .select('airport', 'state')
              .drop_duplicates())
ddf_states.show()

In [ ]:
# 6.
ddf_air_joined = ddf_air.join(ddf_states, on = ['airport'])
ddf_air_joined.show()

In [ ]:
# 7.
ddf_air_weather = (ddf_air_joined.withColumn('weather_condition', sf.when(sf.col('weather_delay') > 1200, 'rainy')
                                                     .when((sf.col('weather_delay') > 1200) & 
                                                           (sf.col('arr_diverted') > 15), 'stormy')
                                                     .otherwise('bright')))
ddf_air_weather.show()

In [ ]:
# 8.
ddf_train = (ddf_air_weather.sort('year', 'month')
                    .limit(20000))
ddf_test = (ddf_air_weather.sort(sf.desc('year', 'month')
                   .limit(8515)))

In [ ]:
# 8a.
(ddf_train, ddf_test) = ddf_air_weather.randomSplit([0.8, 0.2])
ddf_test.count()